In [11]:
import pandas as pd
import os
import re
import collections
import math
import numpy as np
import operator
import random
import math
from math import log
from nltk.stem.snowball import SnowballStemmer
from sklearn.utils import shuffle

In [12]:
stop_words = []
with open('stop_words.txt', encoding='utf-8') as inp:
    m = inp.readlines()
    n = m[0].split(',')
    stop_words.append(n)
stop_words = stop_words[0]
len(stop_words)

315

In [13]:
def spread(arg):
    ret = []
    for i in arg:
        if isinstance(i, list):
            ret.extend(i)
        else:
            ret.append(i)
    return ret

In [14]:
def deep_flatten(arr):
    result = []
    result.extend(
        spread(list(map(lambda x: deep_flatten(x) if type(x) == list else x, arr)))
    )
    return result

In [15]:
def tokenize(text):
    min_length = 3
    text = text.lower()
    pattern = re.compile('[a-zа-я]+')
    result = pattern.findall(text)
    stemmer_ru = SnowballStemmer('russian')
    stemmer_en = SnowballStemmer('english')
    tokens = [stemmer_ru.stem(stemmer_en.stem(word)) for word in result]
    tokens = [word for word in tokens if word not in stop_words]
    filtered_tokens = list(
        filter(lambda token: len(token) >= min_length, tokens)
    )
    return filtered_tokens

In [16]:
def make_dict(t):
    dic = collections.Counter()
    for i in t:
        dic += collections.Counter(i)
    return dict(dic)

In [8]:
train = pd.read_csv('data/news_train.csv', sep='\t')
print(train.shape)

(60000, 3)


In [9]:
theme = np.unique(train['theme'])

In [10]:
print(theme)

['business' 'culture' 'economics' 'forces' 'life' 'media' 'science'
 'sport' 'style' 'travel']


In [11]:
Y = ['business'] * 2099
Y.append(['culture'] * 8405)
Y.append(['economics'] * 8545)
Y.append(['forces'] * 4758)
Y.append(['life'] * 8083)
Y.append(['media'] * 8629)
Y.append(['science'] * 8657)
Y.append(['sport'] * 8510)
Y.append(['style'] * 1148)
Y.append(['travel'] * 1166)
Y = deep_flatten(Y)
Y = pd.DataFrame({'theme': Y})
Y.to_csv('Y.csv', sep=',', encoding='utf-8', index=None)

Нормализация Train и выделение 1000 самых встречающихся слов в каждой теме

In [12]:
Train = []
for k in theme:
    th = train[train['theme'] == k]
    print(th.shape)
    text = th['text'].apply(lambda x: tokenize(x))
    text = list(text)
    dic = make_dict(text)
    newDic = []
    sorted_dic = dict(sorted(dic.items(), key=operator.itemgetter(1))[::-1])
    newDic = list(sorted_dic.keys())[:1000]
    t = []
    for i in text:
        for j in i:
            if j in newDic:
                t.append(j)
        Train.append(t)
        t = []
    print(len(Train))

(2099, 3)
2099
(8405, 3)
10504
(8545, 3)
19049
(4758, 3)
23807
(8083, 3)
31890
(8629, 3)
40519
(8657, 3)
49176
(8510, 3)
57686
(1148, 3)
58834
(1166, 3)
60000


In [13]:
Y = pd.read_csv('Y.csv')

In [14]:
Y.replace(['business', 'culture', 'economics', 'forces', 'life', 'media', 'science', 'sport', 'style','travel'],
          [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)

In [15]:
Train, Y = shuffle(Train, Y, random_state=11)

In [16]:
with open('newTrain.txt', 'w', encoding='utf-8') as out:
    for i in Train:
        for j in i:
            out.write('{},'.format(j))
        out.write('\n')

In [17]:
Y.to_csv('Y.csv', sep=',', encoding='utf-8', index=None)

Нормализация Test

In [18]:
test = pd.read_csv('data/news_test.csv', sep='\t')

In [19]:
test = test['text'].apply(lambda x: tokenize(x))
with open('newTest.txt', 'w', encoding='utf-8') as out:
    for i in list(test):
        for j in i:
            out.write('{},'.format(j))
        out.write('\n')

Представление текста в виде count матрицы 

In [5]:
Train = []
with open('newTrain.txt', encoding='utf-8') as inp:
    for i in inp.readlines():
        nn = i.split(',')
        nn.remove('\n')
        Train.append(nn)

In [6]:
Test = []
with open('newTest.txt', encoding='utf-8') as inp:
    for i in inp.readlines():
        nn = i.split(',')
        nn.remove('\n')
        Test.append(nn)

In [7]:
Y = pd.read_csv('Y.csv')

In [8]:
def conuntVec(doc, dic):
    c = []
    for i in dic:
        c.append(doc.count(i))
    return c

In [9]:
text = Train.copy()
text.extend(Test)
len(text)

75000

In [17]:
dic = make_dict(Train)
print(len(dic))

3520


In [ ]:
dic = dic.keys()

Запись в файл count матрицы

In [ ]:
for doc in Train:
    new_title = conuntVec(doc, dic)
    with open('countVecTrain.txt', 'a', encoding='utf-8') as out:
        for i in list(new_title):
            out.write('{},'.format(i))
        out.write('\n')

In [ ]:
for doc in Test:
    new_title = conuntVec(doc, dic)
    with open('countVecTest.txt', 'a', encoding='utf-8') as out:
        for i in list(new_title):
            out.write('{},'.format(i))
        out.write('\n')

Чтение файлов с матрицей tf

In [29]:
Train = []
with open('countVecTrain.txt', encoding='utf-8') as inp:
    m = inp.readlines()
    for i in m:
        i = i[0:len(i)-2]
        nn = list(map(float, i.split(',')))
        Train.append(nn)

In [30]:
Test = []
with open('countVecTest.txt', encoding='utf-8') as inp:
    m = inp.readlines()
    for i in m:
        i = i[0:len(i)-2]
        nn = list(map(float, i.split(',')))
        Test.append(nn)

In [31]:
Test = pd.DataFrame(Test)
Train = pd.DataFrame(Train)

In [32]:
Y = pd.read_csv('Y.csv')

In [33]:
Train['Y'] = Y

Наивный Байесовский классификатор

In [34]:
def train(Train):
    dic = 3350
    classes = range(1,11)
    freq = []
    clas = []
    for i in classes:
        feat =  np.zeros(dic)
        kol = 0
        l = Train[Train['Y']==i]
        l.drop(['Y'], axis=1, inplace=True)
        l = np.array(l)
        for j in l:
            summ = []
            p = np.array(j)
            for k in range(dic):
                summ.append(p[k]+feat[k])
            feat = summ
            kol += 1
        freq.append(feat)
        clas.append(kol)
    
    
    for i in range(len(freq)):
        for j in range(len(freq[i])):
            freq[i][j] = freq[i][j] / clas[i]
    for c in range(len(clas)):              
        clas[c] = clas[c] / len(Y)
        
    return clas, freq

In [35]:
def classify(clas, feats):
    classes, prob = clas
    pr = []
    csl = []
    for cl in range(len(classes)):
        res = 0
        for feat in range(len(feats)):
            res += -log(prob[cl][feat]*feats[feat] + 10**(-7))
        res += -log(classes[cl])
        pr.append(res)
        csl.append(cl)
    return csl[pr.index(min(pr))]
    return min(classes, key = lambda cl: -log(classes[cl]) + sum(-log(prob[cl]*feat, 10**(-7)) for feat in feats))

In [36]:
classifier = train(Train)

D:\Programs\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [37]:
Y_pr = []
for i in np.array(Test):
    Y_pr.append(classify(classifier, i))

In [38]:
Y_pr = pd.DataFrame(Y_pr)
Y_pr.replace([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
             ['business', 'culture', 'economics', 'forces', 'life', 'media', 'science', 'sport', 'style','travel'],
             inplace=True)
Y_pr.to_csv('Y_pr.csv', header=None, index=None)